In [7]:
import pandas as pd
import numpy as np

from scipy.sparse import csc_matrix

In [10]:
data_dir = '../data/processed/'

In [42]:
rating_df = pd.read_csv(data_dir + 'train_rating.csv')
rating_df.shape

(12800168, 5)

In [46]:
mean_ratings = rating_df.groupby('movieId').rating.mean().reset_index()
mean_ratings.head()

,movieId,rating
0,1,3.920535
1,2,3.209256
2,3,3.146716
3,4,2.856856
4,5,3.070523


In [120]:
movies_df = pd.read_csv(data_dir + 'movie.csv')
# movies_df = movies_df[movies_df.movieId.isin(mean_ratings.movieId)]
movies_df.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,horror,imax,musical,mystery,romance,scifi,thriller,unknown,war,western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
movies_df[['movieId','title']].sample(10)

,movieId,title
14749,73778,"Story of a Cheat, The (Roman d'un tricheur, Le..."
16120,81566,"Stranger in Me, The (Das Fremde in mir) (2008)"
21573,104604,Something Real and Good (2013)
18053,90476,"Little Bit of Heaven, A (2011)"
16916,85565,Chalet Girl (2011)
26855,129015,Secrets of Eden (2012)
14398,72113,Is Anybody There? (2008)
19131,95115,Inauguration of the Pleasure Dome (1954)
15363,78316,Letters to Juliet (2010)
4666,4762,Djomeh (2000)


In [107]:
cross = movies_df.merge(mean_ratings, how='inner', on='movieId')
cross.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,imax,musical,mystery,romance,scifi,thriller,unknown,war,western,rating
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.920535
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.209256
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,3.146716
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,2.856856
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.070523


In [108]:
cross.drop(columns='title', inplace=True)

In [109]:
from sklearn.preprocessing import MinMaxScaler

In [110]:
scaler = MinMaxScaler()
scaler.fit(cross)
features = scaler.transform(cross)

In [111]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(
    # Quantidade de recomendações
    n_neighbors=6,
    # Algoritmo
    algorithm='brute',
    # metrica de distancia
    metric='minkowski'
)

model.fit(features)

NearestNeighbors(algorithm='brute', n_neighbors=6)

In [116]:
sample = cross[cross.movieId == 1]
sample

,movieId,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,...,imax,musical,mystery,romance,scifi,thriller,unknown,war,western,rating
0,1,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3.920535


In [117]:
distances, indexes = model.kneighbors(sample)
indexes = indexes[0]
indexes

/home/luanabarros/miniconda3/envs/filmes_pra_ti/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


array([24890, 22879, 23107, 18724, 23105, 23358])

In [118]:
recs = cross.loc[indexes].merge(movies_df[['movieId','title']], on='movieId', how='inner')

In [119]:
recs

,movieId,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,...,musical,mystery,romance,scifi,thriller,unknown,war,western,rating,title
0,131248,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,4.000000,Brother Bear 2 (2006)
1,114552,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,3.403226,"Boxtrolls, The (2014)"
2,115879,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,3.400000,Toy Story Toons: Small Fry (2011)
3,95311,0,0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,4.061111,Presto (2008)
4,115875,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,3.312500,Toy Story Toons: Hawaiian Vacation (2011)
5,117486,0,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,4.000000,A Connecticut Yankee in King Arthur's Court (1...
